# Laboratório 9: Populações de Urso

### Referente ao capítulo 15 

O cenário neste laboratório é motivado pelo *Great Smoky Mountain National Park*. É considerado a população de ursos em um parque genérico em áreas populadas por humanos, tal que essas regiões apresentem bordas compartilhadas. A emigração e imigração de ursos é baseada na conectividade entre caminhos. 

|Parâmetro|Significado|
|---------|-----------|
|$r$|taxa de crecimento de ursos|
|$K$|capacidade de carga|
|$m_p$|proporção da fronteira do parque conectada com a floresta|

Seja $P(t), F(t)$ e $O(t)$ a densidade de ursos no parque, na floresta e nas regiões de fora. Os controles serão $u_p$ e $u_f$, que são as taxas de caça de ursos no parque e na floresta, respectivamente. 

Atualmente a caça em parques não é legalizada, mas o modelo considera que ocorrem também, além da caça na floresta. Queremos minimizar o número de ursos nas áreas populadas por humanos. Assim, o problema pode ser descrito pelo seguinte modelo. 


$$ \min_{u_p, u_f} \int_0^T O(t) + c_pu_p(t)^2 + c_fu_f(t)^2 dt $$

$$\text{sujeito a  }P'(t) = rP(t) - \frac{r}{K}P(t)^2 + \frac{m_fr}{K}\left(1 - \frac{P(t)}{K}\right)F(t)^2 - u_p(t)P(t),$$
$$F'(t) = rF(t) - \frac{r}{K}F(t)^2 + \frac{m_pr}{K}\left(1 - \frac{F(t)}{K}\right)P(t)^2 - u_f(t)F(t),$$
$$O'(t) = r(1 - m_p)\frac{P(t)^2}{K} + r(1 - m_f)\frac{F(t)^2}{K} + \frac{m_fr}{K^2}P(t)F(t)^2 + \frac{m_pr}{K^2}P(t)^2F(t),$$

$$P(0) = P_0 \geq 0, F(0) = F_0 \geq 0, O(0) = O_0 \geq 0, $$
$$0 \leq u_p(t) \leq 1, 0 \leq u_f(t) \leq 1 $$

O termo $\frac{r}{K}P(t)^2$ representa a emigração densidade-depentente. O terceiro termo na primeira equação é a imigração da floresta densidade-dependente e $u_pP$ o nível de caça. O primeiro termo da terceira equação é a emigração do parque para regiões fora do parque, enquanto o segundo é a emigração da floresta para outras regiões. 

### Importanto as bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sympy as sp

import sys  
sys.path.insert(0, '../pyscripts/')

from optimal_control_class import OptimalControl

## Condições Necessárias

Vamos utilizar a bilbioteca `sympy` para nos ajudar a escrever as condições necessárias. 

In [2]:
P, F, O, uf, up, l1, l2, l3, cp, cf, r, K, mf, mp, t = sp.symbols('P F O uf up l1 l2 l3 cp cf r K mf mp t')

### O Hamiltoniano

\begin{multline}
H(t, \vec{x}, \vec{u}, \vec{\lambda}) = O(t) + c_pu_p(t)^2 + c_fu_f(t)^2 +  \\ 
\lambda_1(t)\left[rP(t) - \frac{r}{K}P(t)^2 + \frac{m_fr}{K}\left(1 - \frac{P(t)}{K}\right)F(t)^2 - u_p(t)P(t)\right] + \\ 
\lambda_2(t)\left[rF(t) - \frac{r}{K}F(t)^2 + \frac{m_pr}{K}\left(1 - \frac{F(t)}{K}\right)P(t)^2 - u_f(t)F(t)\right] + \\ 
\lambda_3(t)\left[r(1 - m_p)\frac{P(t)^2}{K} + r(1 - m_f)\frac{F(t)^2}{K} + \frac{m_fr}{K^2}P(t)F(t)^2 + \frac{m_pr}{K^2}P(t)^2F(t)\right]
\end{multline}

In [7]:
H =  O + cp*up**2 + cf*uf**2 + l1*(r*P - r*P**2/K + mf*(r/K)*(1 - P/K)*F**2 - up*P) + \
                               l2*(r*F - r*F**2/K + mp*(r/K)*(1 - F/K)*P**2 - uf*F) + \
                               l3*(r*(1 - mp)*P**2/K + r*(1 - mf)*F**2/K + mf*r*P*F**2/K**2 + mp*r*P**2*F/K**2)

### Condições do Estado 

In [8]:
print(sp.diff(H,l1), '\n')
print(sp.diff(H,l2), '\n')
print(sp.diff(H,l3))

F**2*mf*r*(1 - P/K)/K + P*r - P*up - P**2*r/K 

-F**2*r/K + F*r - F*uf + P**2*mp*r*(-F/K + 1)/K 

F**2*r*(1 - mf)/K + F**2*P*mf*r/K**2 + F*P**2*mp*r/K**2 + P**2*r*(1 - mp)/K


### Equações Adjuntas

In [9]:
dl1 = (-1)*sp.diff(H,P)
dl2 = (-1)*sp.diff(H,F)
dl3 = (-1)*sp.diff(H,O)
print(dl1, '\n')
print(dl2, '\n')
print(dl3)

-l1*(-F**2*mf*r/K**2 + r - up - 2*P*r/K) - l3*(F**2*mf*r/K**2 + 2*F*P*mp*r/K**2 + 2*P*r*(1 - mp)/K) - 2*P*l2*mp*r*(-F/K + 1)/K 

-2*F*l1*mf*r*(1 - P/K)/K - l2*(-2*F*r/K + r - uf - P**2*mp*r/K**2) - l3*(2*F*r*(1 - mf)/K + 2*F*P*mf*r/K**2 + P**2*mp*r/K**2) 

-1


### Condições de Otimalidade

In [11]:
du1 = sp.diff(H,up)
du2 = sp.diff(H,uf)
print(du1, '\n')
print(du2)

-P*l1 + 2*cp*up 

-F*l2 + 2*cf*uf


$$ \frac{\partial H}{\partial u_p} > 0 \implies u_p^*(t) = 0 \implies \frac{P\lambda_1}{2c_p} < 0$$
$$ \frac{\partial H}{\partial u_p} < 0 \implies u_p^*(t) = 1 \implies \frac{P\lambda_1}{2c_p} > 1$$
$$ \frac{\partial H}{\partial u_p} = 0 \implies 0 \leq u_p^*(t) = \frac{P\lambda_1}{2c_p} \leq 1$$

$$u_p^* = \max(0, \min(1, \frac{P\lambda_1}{2c_p}))$$

$$ \frac{\partial H}{\partial u_f} > 0 \implies u_f^*(t) = 0 \implies \frac{F\lambda_2}{2c_f} < 0$$
$$ \frac{\partial H}{\partial u_f} < 0 \implies u_f^*(t) = 1 \implies \frac{F\lambda_2}{2c_f} > 1$$
$$ \frac{\partial H}{\partial u_f} = 0 \implies 0 \leq u_f^*(t) = \frac{F\lambda_2}{2c_f} \leq 1$$

$$u_f^* = \max(0, \min(1, \frac{F\lambda_2}{2c_f}))$$

## Aplicando a classe ao exemplo 

Nesse laboratório, o leitor é convidado a escrever as equações e realizar os testes por conta. Você deve mudar a célula de Raw para Code e escrever as funções como fizemos até então. 